## Swedcham Debit Note Automation

1. Read American Credit Card Invoice
2. Create a new Word File with the current Debit Note
3. Generate a PDF from the Word File
4. Send a mail to Swedcham

#### 1. Read American Credit Card Invoice

In [9]:
# Import Libraries
import PyPDF2
import datetime
import os
import re

In [4]:
# Get American Credit Card Invoice folder
cc_folder = 'D:\\Primeway\\Users Pw\\Oscar.Carvalho\\Contas\\Cartões de Crédito\\American\\'
pdf_pass = 'LaylaLaylaLayla'

In [3]:
# Get last invoice file name

# Get current year
today = datetime.date.today()
year = str(today.year)
# print(year)

path = cc_folder + year + '\\'

file_list = [entry.name for entry in os.scandir(path) if entry.name[0:4] == year]
filename = file_list[len(file_list)-1]

print(path)
print(filename)

D:\Primeway\Users Pw\Oscar.Carvalho\Contas\Cartões de Crédito\American\2023\
2023-07-Extrato-American-Bradesco.pdf


In [27]:
# Open PDF file and get Mailchimp line
filename = '2023-07-Extrato-American-Bradesco.pdf'

with open(path + filename, 'rb') as obj:
   pdfReader = PyPDF2.PdfReader(obj)
   if pdfReader.is_encrypted:
      pdfReader.decrypt(pdf_pass)
   page_txt = pdfReader.pages[0].extract_text()

mailchimp_line = re.search("MAILCHIMP.+\\n", page_txt)
if (not mailchimp_line == None):
   	print(mailchimp_line.group())
else:
   print('Mailchimp line not found!')

MAILCHIMP 55,00 USD 55,00 5,0300 276,65

